In [1]:
import logging
import numpy as np
import random
from math import inf
from itertools import chain
from typing import Callable
from gx_utils import *

logging.basicConfig(format="%(message)s", level=logging.INFO)

## Graph search for the Set Covering problem

In [2]:
class State:
    def __init__(self, data: list):
        self._list = sorted(data.copy())
        self.set_covered = set(chain(*self._list))

    def __hash__(self):
        return hash(tuple(chain(*self._list)))

    def __eq__(self, other):
        return tuple(self.set_covered) == tuple(other.set_covered)

    def __contains__(self, other):
        return set(other) in self.set_covered

    def __le__(self, other):
        return self.set_covered <= other.set_covered

    def __lt__(self, other):
        return self.set_covered < other.set_covered

    def __str__(self):
        return str(chain(*self._list))

    def __repr__(self):
        return repr(self._list)

    def covers(self, other: list):
        return set(other) <= self.set_covered

    @property
    def data(self):
        return self._list

    def copy_data(self):
        return self._list.copy()

In [3]:
def goal_test(state):
    return state.set_covered == goal

In [4]:
def possible_actions(state: State):
    return (l for l in all_lists if not state.covers(l))

In [5]:
def result(state, action):
    current_list = state.copy_data()
    current_list.append(action)
    return State(current_list)

In [6]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

## Generalized search algorithm - find minimum (reviewers ignore this)

In [8]:
def search_min(
    initial_state: State,
    goal_test: Callable,
    parent_state: dict,
    state_cost: dict,
    priority_function: Callable,
    unit_cost: Callable,
):
    frontier = PriorityQueue()
    parent_state.clear()
    state_cost.clear()

    state = initial_state
    parent_state[state] = None
    state_cost[state] = 0

    min_cost = inf
    min_state = None

    i = 0
    n_frontier = 0
    
    while state is not None:
        logging.debug(f'i = {i}')
        logging.debug(f'Current state -> {state.data}')

        if goal_test(state):
            logging.debug(f'Found a solution: {state.data}')
            if state_cost[state] < min_cost:
                logging.debug(f'Updating min cost -> {state_cost[state]}')
                min_cost = state_cost[state]
                min_state = state
                logging.info(f'New best solution: w = {min_cost} steps = {len(state.data)} (visited {i} nodes)')
        else:
            for a in possible_actions(state):
                new_state = result(state, a)
                cost = unit_cost(a)
                if new_state not in state_cost and new_state not in frontier:
                    parent_state[new_state] = state
                    state_cost[new_state] = state_cost[state] + cost
                    frontier.push(new_state, p=priority_function(new_state))
                    n_frontier += 1
                    logging.debug(f"Added new node ({n_frontier}) to frontier (cost={state_cost[new_state]}) -> {new_state.data}")

        if frontier:
            state = frontier.pop()
        else:
            state = None
        
        i += 1

    logging.debug(f'Total nodes in frontier: {n_frontier}')

    path = list()
    s = min_state
    while s:
        path.append(s.copy_data())
        s = parent_state[s]

    logging.info(f'Done in {i} iterations of the main loop')

    logging.debug('Min path followed:')
    logging.debug(list(enumerate(reversed(path))))

    return min_state

## Generalized search algorithm for finding the first solution

In [7]:
def search(
    initial_state: State,
    goal_test: Callable,
    parent_state: dict,
    state_cost: dict,
    priority_function: Callable,
    unit_cost: Callable,
):
    frontier = PriorityQueue()
    parent_state.clear()
    state_cost.clear()

    state = initial_state
    parent_state[state] = None
    state_cost[state] = 0

    i = 0
    n_frontier = 0
    
    while state is not None and not goal_test(state):
        logging.debug(f'i = {i}')
        logging.debug(f'Current state -> {state.data}')
        for a in possible_actions(state):
            new_state = result(state, a)
            cost = unit_cost(a)
            if new_state not in state_cost and new_state not in frontier:
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                frontier.push(new_state, p=priority_function(new_state))
                n_frontier += 1
                logging.debug(f"Added new node ({n_frontier}) to frontier (cost={state_cost[new_state]}) -> {new_state.data}")

        if frontier:
            state = frontier.pop()
        else:
            state = None
        
        i += 1

    logging.debug(f'Total nodes in frontier: {n_frontier}')

    path = list()
    s = state
    while s:
        path.append(s.copy_data())
        s = parent_state[s]
    
    logging.info(f'Visited {i:,} nodes')

    logging.debug('Path followed:')
    logging.debug(list(enumerate(reversed(path))))

    return state

## Breadth-First

In [22]:
logging.getLogger().setLevel(logging.INFO)

for N in [5, 10, 20]:
    logging.info(f'N = {N}')
    goal = set(range(N))
    initial_state = State(list())

    all_lists = problem(N, seed=42)

    all_lists = [list(t) for t in set(tuple(_) for _ in all_lists)] # Remove duplicates

    parent_state = dict()
    state_cost = dict()

    solution = search(
        initial_state,
        goal_test=goal_test,
        parent_state=parent_state,
        state_cost=state_cost,
        priority_function=lambda s: len(state_cost),
        unit_cost=lambda a: len(a),
    )

    logging.info(
        f"Found solution for N={N}: w={sum(len(_) for _ in solution.data)} (steps={len(solution.data)}) (bloat={(sum(len(_) for _ in solution.data)-N)/N*100:.0f}%)"
    )

N = 5
Visited 48 nodes
Found solution for N=5: w=6 (steps=3) (bloat=20%)
N = 10
Visited 1,001 nodes
Found solution for N=10: w=11 (steps=3) (bloat=10%)
N = 20
Visited 6,587 nodes
Found solution for N=20: w=29 (steps=4) (bloat=45%)


## Depth-First

In [23]:
logging.getLogger().setLevel(logging.INFO)

for N in [5, 10, 20, 100, 500, 1000]:
    logging.info(f'N = {N}')
    goal = set(range(N))
    initial_state = State(list())

    all_lists = problem(N, seed=42)

    all_lists = [list(t) for t in set(tuple(_) for _ in all_lists)] # Remove duplicates

    parent_state = dict()
    state_cost = dict()

    solution = search(
        initial_state,
        goal_test=goal_test,
        parent_state=parent_state,
        state_cost=state_cost,
        priority_function=lambda s: -len(state_cost),
        unit_cost=lambda a: len(a),
    )

    logging.info(
        f"Found solution for N={N}: w={sum(len(_) for _ in solution.data)} (steps={len(solution.data)}) (bloat={(sum(len(_) for _ in solution.data)-N)/N*100:.0f}%)"
    )

N = 5
Visited 4 nodes
Found solution for N=5: w=6 (steps=4) (bloat=20%)
N = 10
Visited 6 nodes
Found solution for N=10: w=16 (steps=6) (bloat=60%)
N = 20
Visited 8 nodes
Found solution for N=20: w=53 (steps=8) (bloat=165%)
N = 100
Visited 11 nodes
Found solution for N=100: w=339 (steps=11) (bloat=239%)
N = 500
Visited 18 nodes
Found solution for N=500: w=2679 (steps=18) (bloat=436%)
N = 1000
Visited 16 nodes
Found solution for N=1000: w=4880 (steps=16) (bloat=388%)


## A*

In [8]:
def h1(state: State):
    return len(list(chain(*state.data)))

def h2(state: State):
    c = Counter(list(chain(*state.data)))
    return c.total() - len(list(c))

In [9]:
def tup_priority_f(new_state: State):
    # Only difference with h2 is that this returns a tuple. First element is the same (the total bloat) and second
    # is the negative of the number of distinct elements in the new_state

    c = Counter(list(chain(*new_state.data)))
    return (c.total() - len(list(c)), -sum(c[e] == 1 for e in c))


In [22]:
logging.getLogger().setLevel(logging.INFO)

for N in [5, 10, 20]:
# for N in [20]:
    logging.info(f'N = {N}')
    goal = set(range(N))
    initial_state = State(list())

    all_lists = problem(N, seed=42)

    all_lists = [list(t) for t in set(tuple(_) for _ in all_lists)] # Remove duplicates

    parent_state = dict()
    state_cost = dict()

    solution = search(
        initial_state,
        goal_test=goal_test,
        parent_state=parent_state,
        state_cost=state_cost,
        priority_function=h2,
        unit_cost=lambda a: len(a),
    )

    logging.info(
        f"Found solution for N={N}: w={sum(len(_) for _ in solution.data)} (steps={len(solution.data)}) (bloat={(sum(len(_) for _ in solution.data)-N)/N*100:.0f}%)"
    )

N = 5
Visited 40 nodes
Found solution for N=5: w=5 (steps=3) (bloat=0%)
N = 10
Visited 975 nodes
Found solution for N=10: w=10 (steps=5) (bloat=0%)
N = 20
Visited 3,560 nodes
Found solution for N=20: w=23 (steps=5) (bloat=15%)
